In [141]:
%run GraphFamilies.ipynb
from IPython.display import clear_output

In [148]:
# Get list of graphs from graph families

#SETTINGS:
# Higher settings will increase accuracy but result in slower runtime

# Maximum size hole to include
maxhole = 12

# Maximum length path to include
maxpath = 6

# TRUE: Use connected graph families
# FALSE: Use disconnected graph familise
connected = True 

# Mode:
#mode = "induced_subgraph"
mode = "subgraph"
#mode = "induced_minor"
#mode = "minor"


#############
graph_family_list = AllFamilies(mode, maxhole, maxpath, connected)
forbidden_graphs = [[],[]]
print(f"{len(graph_family_list)} graphs included for testing")

315 graphs included for testing


In [149]:
# Helper functions

def get_all_induced_subgraphs(G):
    G_sg_list = []

    for vertex_subset in Subsets(G.vertices()):
            current_sg = G.subgraph(vertex_subset)

            add_flag = True
            for H in G_sg_list:
                if (current_sg.order() != H.order()) or (current_sg.size() != H.size()): #Different number of vertices/edges, no need to test isomorphism
                    continue
                elif H.is_isomorphic(current_sg):
                    add_flag = False
                    break 
            
            if add_flag == True:
                G_sg_list.append(current_sg)
    
    return G_sg_list

def get_all_subgraphs(G):
    G_sg_list = []

    for vertex_subset in Subsets(G.vertices()):
        current_i_sg = G.subgraph(vertex_subset)
        for edge_subset in Subsets(current_i_sg.edges()):
            current_sg = G.subgraph(vertices=vertex_subset, edges=edge_subset)

            add_flag = True
            for H in G_sg_list:
                if (current_sg.order() != H.order()) or (current_sg.size() != H.size()): #Different number of vertices/edges, no need to test isomorphism
                    continue
                elif H.is_isomorphic(current_sg):
                    add_flag = False
                    break 
            
            if add_flag == True:
                G_sg_list.append(current_sg)
    
    return G_sg_list

def get_all_induced_minors(G):
    G_sg_list = []

    for vertex_subset in Subsets(G.vertices()):
        current_i_sg = G.subgraph(vertex_subset)
        for edge_subset in Subsets(current_i_sg.edges()):
            current_i_m = deepcopy(current_i_sg)
            current_i_m.allow_loops(True); current_i_m.allow_multiple_edges(True)
            current_i_m.contract_edges(edge_subset)
            current_i_m.remove_loops(); current_i_m.remove_multiple_edges()
            current_i_m.allow_loops(False); current_i_m.allow_multiple_edges(False)

            add_flag = True
            for H in G_sg_list:
                if (current_i_m.order() != H.order()) or (current_i_m.size() != H.size()): #Different number of vertices/edges, no need to test isomorphism
                    continue
                elif H.is_isomorphic(current_i_m):
                    add_flag = False
                    break 
            
            if add_flag == True:
                G_sg_list.append(current_i_m)
    
    return G_sg_list

def get_all_minors(G):
    G_sg_list = []

    for vertex_subset in Subsets(G.vertices()):
        current_i_sg = G.subgraph(vertex_subset)
        for edge_subset in Subsets(current_i_sg.edges()):
            current_sg = G.subgraph(vertices=vertex_subset, edges=edge_subset)
            for sg_edge_subset in Subsets(current_i_sg.edges()):
                current_m = deepcopy(current_sg)
                current_m.allow_loops(True); current_m.allow_multiple_edges(True)
                current_m.contract_edges(sg_edge_subset)
                current_m.remove_loops(); current_m.remove_multiple_edges()
                current_m.allow_loops(False); current_m.allow_multiple_edges(False)

                add_flag = True
                for H in G_sg_list:
                    if (current_m.order() != H.order()) or (current_m.size() != H.size()): #Different number of vertices/edges, no need to test isomorphism
                        continue
                    elif H.is_isomorphic(current_m):
                        add_flag = False
                        break 
                
                if add_flag == True:
                    G_sg_list.append(current_m)
    
    return G_sg_list

def get_best_sgs(graph_list = graph_family_list):
    # Need only to test graphs know contained in graph with the least subgraphs
    # Estimate using number of vertices
    # Find this graph
    best_order = 1000000
    for i in graph_list:
        current_order = i.order()
        if current_order < best_order:
            best_order = current_order
            best_graph = i
    
    return best_graph

def remove_redundant(tuple_list):
    cleaned_list = deepcopy(tuple_list)

    for i in tuple_list:
        for j in tuple_list:
            if tuple_list.index(i) == tuple_list.index(j): continue

            i_checklist = 0
            for x in i:
                for y in j:
                    if multi_relation_search(x,y) is not None:
                        i_checklist = i_checklist + 1
                        break
            
            if i_checklist == len(i):
                if j in cleaned_list: cleaned_list.remove(j)

    
    return cleaned_list

def check_exclusions(check_tuple, exclusion_list):

    for i in exclusion_list:
        i_checklist = 0

        for x in i:
            for y in check_tuple:
                if multi_relation_search(x,y) is not None:
                    i_checklist = i_checklist + 1
                    break
        
        if i_checklist == len(i):
            return True
    
    return False

def multi_relation_search(G,H):
    match mode:
        case "induced_subgraph": 
            return G.subgraph_search(H, induced=True)
        case "subgraph": 
            return G.subgraph_search(H, induced=False)
        case "induced_minor":
            try:
                output = G.minor(H, induced=1)
            except ValueError:
                return None
            else:
                return output
        case "minor":
            try:
                output = G.minor(H, induced=0)
            except ValueError:
                return None
            else:
                return output



In [159]:
# Find classification for single induced subgraph

def find_forbidden_1(graph_list = graph_family_list,exclusions=[]):
    forbidden_graphs = []
    match mode:
        case "induced_subgraph": test_graph_list = get_all_induced_subgraphs(get_best_sgs(graph_list))
        case "subgraph": test_graph_list = get_all_subgraphs(get_best_sgs(graph_list))
        case "induced_minor": test_graph_list = get_all_induced_minors(get_best_sgs(graph_list))
        case "minor": test_graph_list = get_all_minors(get_best_sgs(graph_list))
    
    # Order list of subgraphs we need to test by most vertices first, followed by most edges first
    test_graph_list.sort(reverse = True, key=lambda element: (Graph.order(element), Graph.size(element)))

    i=0 

    # Check each 
    for test_sg in test_graph_list:
        i = i + 1; print(f"\r {1,i,len(test_graph_list)}..................",end="")

        add_flag = True

        # Make sure graph isn't excluded
        for exclusion in exclusions:
            if multi_relation_search(exclusion,test_sg) is not None:
                add_flag = False
                break
        if add_flag == False: continue
        
        # Make sure subgraph isn't already contained in a graph already included (most vertices first means this test is simple)
        for already_seen in forbidden_graphs:
            if multi_relation_search(already_seen,test_sg) is not None:
                add_flag = False
                break
        if add_flag == False: continue
        
        # Look at each graph in each family
        for current_graph in graph_list:
            if multi_relation_search(current_graph,test_sg) is None:
                add_flag = False
                break
            
        
        # Include subgraph only if found as a subgraph in all graphs
        if add_flag == True:
            forbidden_graphs.append(test_sg)
    
    return(forbidden_graphs)

In [160]:
def find_forbidden_k(k,graph_list = graph_family_list,exclusions=[]):
    forbidden_graph_k = []
    
    match mode:
        case "induced_subgraph": test_graph_list = get_all_induced_subgraphs(get_best_sgs(graph_list))
        case "subgraph": test_graph_list = get_all_subgraphs(get_best_sgs(graph_list))
        case "induced_minor": test_graph_list = get_all_induced_minors(get_best_sgs(graph_list))
        case "minor": test_graph_list = get_all_minors(get_best_sgs(graph_list))
    
    
    # Order list of subgraphs we need to test by most vertices first, followed by most edges first
    test_graph_list.sort(reverse = True, key=lambda element: (Graph.order(element), Graph.size(element)))

    i = 0

    # Check each 
    for test_sg in test_graph_list:
        i = i + 1; print(f"\r {k,i,len(test_graph_list)}..................",end="")
        add_flag = True

        # Check that isn't in the monogenic case
        for case_1 in forbidden_graphs[1]:
            if multi_relation_search(case_1,test_sg) is not None:
                add_flag = False
                break
        if add_flag == False: continue

        # Create a new list by removing all graphs containing our subgraph
        culled_graph_list = deepcopy(graph_list)
        for current_graph in graph_list:
            good_flag = False

            # Check if the subgraph we're testing appears as a subgraph in this graph
            if multi_relation_search(current_graph,test_sg) is not None:
                good_flag = True
            
            # If it does, remove it from our new list
            if good_flag == True:
                culled_graph_list.remove(current_graph)
        
        # Find graph (k-1)-tuples which are in all graphs of this new table but not in the monogenic case
        if culled_graph_list == []: continue
        if k - 1 == 1:
            second_graphs = find_forbidden_1(culled_graph_list,forbidden_graphs[k-1])
        else:
            second_graphs = find_forbidden_k(k-1,culled_graph_list,forbidden_graphs[k-1])

        # Add ts
        for second_graph in second_graphs:
            t = [test_sg]
            if k == 2:
                t.append(second_graph)
            else:
                for H in second_graph:
                    t.append(H)
            if check_exclusions(t, exclusions) == False: forbidden_graph_k.append(t)

    forbidden_graph_k_cleaned = remove_redundant(forbidden_graph_k)  

    return forbidden_graph_k_cleaned


In [ ]:
f1 = open(r"1_case_sg.txt","w")
for case in forbidden_graphs[1]:
    f1.write(case.graph6_string())
    f1.write("\n")
f1.close()
print(len(forbidden_graphs[1]))

f2 = open(r"2_case_sg.txt","w")
for case in forbidden_graphs[2]:
    f2.write(f"{case[0].graph6_string()},{case[1].graph6_string()}")
    f2.write("\n")
f2.close()
print(len(forbidden_graphs[2]))

f3 = open(r"3_case_sg.txt","w")
for case in forbidden_graphs[3]:
    f3.write(f"{case[0].graph6_string()},{case[1].graph6_string()},{case[2].graph6_string()}")
    f3.write("\n")
f3.close()
print(len(forbidden_graphs[3]))

f4 = open(r"4_case_sg.txt","w")
for case in forbidden_graphs[4]:
    f4.write(f"{case[0].graph6_string()},{case[1].graph6_string()},{case[2].graph6_string()},{case[3].graph6_string()}")
    f4.write("\n")
f4.close()
print(len(forbidden_graphs[4]))

f5 = open(r"5_case.txt","w")
for case in forbidden_graphs[5]:
    f5.write(f"{case[0].graph6_string()},{case[1].graph6_string()},{case[2].graph6_string()},{case[3].graph6_string()},{case[4].graph6_string()}")
    f5.write("\n")
f5.close()
